# VK API

In [ ]:
!pip install vk_api -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 KB 4.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import vk_api
import requests
import os
import json
import random
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [ ]:
TOKEN = 'vk1.a.IF7DTk4wskl4H8wl3ZTIp_r96xmv-OBEe4NumVHFRyBwxCXAXr1uIZqAcFnncSDsszo5AfuF0zQ-9SJZthJ_71ZWlCc7tV0KGLBZh3lMLykkT9mrhk4seHtMxcby8oDaqB5fwYkSIBvhz1xYcm_RYV9xYedRS5o8g7_uClcuG8KZ3S_7TB50iTWOdZV9NnHCixhaVOZbv9DmLyMPWITf2Q'

In [ ]:
vk_session = vk_api.VkApi(token=TOKEN)

In [ ]:
vk = vk_session.get_api()

In [ ]:
vk.messages.send(chat_id=1, random_id=1, message='Wake up, Neo')

0

In [ ]:
# vk.groups.get(user_id=)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Karpov_Courses/data/lesson_5_ads_data.csv.zip')

In [ ]:
df.head()

,ad_id,time,event,date,ad_cost_type,has_video,client_union_id,campaign_union_id,platform,ad_cost,target_audience_count,user_id
0,23456,1554076848,view,2019-04-01,CPM,0,5681,23456,android,190.0,125560,793
1,111941,1554135866,view,2019-04-01,CPM,0,111793,111941,ios,215.2,32277,1041
2,111941,1554135866,view,2019-04-01,CPM,0,111793,111941,ios,215.2,32277,41
3,42987,1554135866,view,2019-04-01,CPM,0,28541,42985,ios,199.1,80160,898
4,23456,1554135866,view,2019-04-01,CPM,0,5681,23456,web,190.0,125560,29


In [ ]:
ad_data = df.groupby(['ad_id', 'event'], as_index=False).agg({'user_id': 'count'})

In [ ]:
ad_data = ad_data.pivot(index='ad_id', columns='event', values='user_id').reset_index()

In [ ]:
ad_data = ad_data.fillna(0).assign(ctr=ad_data['click'] / ad_data['view'])

In [ ]:
ad_data.head()

event,ad_id,click,view,ctr
0,3,9.0,490.0,0.018367
1,2132,1.0,95.0,0.010526
2,2276,2.0,1454.0,0.001376
3,2475,0.0,132.0,NaN
4,2643,3.0,286.0,0.010490


In [ ]:
top_ctr = ad_data.query('click > 20 & view > 100').sort_values('ctr').tail(10)

In [ ]:
top_ctr.to_csv('top_ctr_data.csv', index=False)

In [ ]:
path = '/content/top_ctr_data.csv'
file_name = 'top_ctr_data.csv'

path_to_file = path
upload_url = vk.docs.getMessagesUploadServer(peer_id=2000000001)['upload_url']
file = {'file': (file_name, open(path_to_file, 'rb'))}

response = requests.post(upload_url, files=file)

json_data = json.loads(response.text)
json_data

{'file': '513613|0|-1|237031|c60e5807be|csv|403|top_ctr_data.csv|c2f9e346fc49058f43500d24dbf7ae44|327e9a9202866af4cdc7fa674f01ef58||||eyJkaXNrIjo1MywicGVlcl9zZW5kZXIiOiItMjE5ODczNDQ5In0='}

In [ ]:
saved_file = vk.docs.save(file=json_data['file'], title=file_name)

In [ ]:
saved_file

{'type': 'doc',
 'doc': {'id': 657076553,
  'owner_id': 513613,
  'title': 'top_ctr_data.csv',
  'size': 403,
  'ext': 'csv',
  'date': 1681128895,
  'type': 1,
  'url': 'https://vk.com/doc513613_657076553?hash=rWRuhvBNsMHx2Ab0Uechv08hpN54yeIEm198tqzpGF0&dl=FUZDCOJYG4ZTINBZ:1681128895:tAGclTz9BYyHydVGeYK587iaf6HnQTJm3GYPTJgjNBH&api=1&no_preview=1',
  'is_unsafe': 0}}

In [ ]:
attachment = f"doc{saved_file['doc']['owner_id']}_{saved_file['doc']['id']}"

In [ ]:
vk.messages.send(chat_id=1, random_id=3, message='Топ объявлений по CTR', attachment=attachment)

0

# ЯндексМетрика API

In [ ]:
url = 'https://api-metrika.yandex.net/stat/v1/data?'
visits = 'metrics=ym:s:visits&dimensions=ym:s:date&id=44147844'

In [ ]:
visits_url = url + visits

In [ ]:
visits_request = requests.get(visits_url)

In [ ]:
visits_request

<Response [200]>

In [ ]:
json_data = json.loads(visits_request.text)

In [ ]:
json_data['data']

[{'dimensions': [{'name': '2023-04-04'}], 'metrics': [949.0]},
 {'dimensions': [{'name': '2023-04-06'}], 'metrics': [843.0]},
 {'dimensions': [{'name': '2023-04-05'}], 'metrics': [838.0]},
 {'dimensions': [{'name': '2023-04-07'}], 'metrics': [727.0]},
 {'dimensions': [{'name': '2023-04-09'}], 'metrics': [378.0]},
 {'dimensions': [{'name': '2023-04-08'}], 'metrics': [349.0]},
 {'dimensions': [{'name': '2023-04-10'}], 'metrics': [331.0]}]

In [ ]:
pd.DataFrame([(i['dimensions'][0]['name'], i['metrics'][0]) for i in json_data['data']], columns=['date', 'visits'])

,date,visits
0,2023-04-04,949.0
1,2023-04-06,843.0
2,2023-04-05,838.0
3,2023-04-07,727.0
4,2023-04-09,378.0
5,2023-04-08,349.0
6,2023-04-10,331.0


In [ ]:
[(i['dimensions'][0]['name'], 
  i['metrics'][0]) for i in json_data['data']]

[('2023-04-04', 949.0),
 ('2023-04-06', 843.0),
 ('2023-04-05', 838.0),
 ('2023-04-07', 727.0),
 ('2023-04-09', 378.0),
 ('2023-04-08', 349.0),
 ('2023-04-10', 331.0)]

In [ ]:
!pip install yadisk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 KB 5.9 MB/s eta 0:00:00


In [ ]:
import yadisk